In [235]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [236]:

# 데이터 로드 및 전처리
digits = load_digits()


In [237]:
digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [238]:
X = digits.data.astype(np.float32)
y = digits.target.astype(np.int64)

In [239]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]], dtype=float32)

In [240]:
y

array([0, 1, 2, ..., 8, 9, 8], dtype=int64)

In [241]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15/0.85, random_state=42)

In [242]:

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)  # 분류용 레이블은 int64(LongTensor)로
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_test  = torch.tensor(y_test,  dtype=torch.int64)
X_val   = torch.tensor(X_val,   dtype=torch.float32)
y_val   = torch.tensor(y_val,   dtype=torch.int64)

In [243]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

(torch.Size([1257, 64]),
 torch.Size([270, 64]),
 torch.Size([270, 64]),
 torch.Size([1257]),
 torch.Size([270]),
 torch.Size([270]))

In [244]:
# DataLoader 정의 (배치 크기 32)
train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=32, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=32, shuffle=False
)
test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=32, shuffle=False
)

In [245]:


class DigitDNN(nn.Module):
    def __init__(self):
        super(DigitDNN, self).__init__()
        self.fc1 = nn.Linear(64, 80)
        self.fc2 = nn.Linear(80, 80)
        self.fc3 = nn.Linear(80, 64)
        self.fc4 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DigitDNN()


# 손실함수 및 최적화 기법 설정
criterion = nn.CrossEntropyLoss()         # 다중 클래스 분류용
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [246]:
num_epochs = 30

for epoch in range(1, num_epochs + 1):
    # ——— Training ———
    model.train()
    train_loss = 0.0
    train_correct = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        preds = outputs.argmax(dim=1)
        train_correct += (preds == y_batch).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  = train_correct / len(train_loader.dataset)

    # ——— Validation ———
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * X_batch.size(0)
            preds = outputs.argmax(dim=1)
            val_correct += (preds == y_batch).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc  = val_correct / len(val_loader.dataset)

    print(f"Epoch {epoch:2d}/{num_epochs} — "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


Epoch  1/30 — Train Loss: 1.5293, Acc: 0.5839 | Val Loss: 0.6372, Acc: 0.8741
Epoch  2/30 — Train Loss: 0.3797, Acc: 0.9173 | Val Loss: 0.2643, Acc: 0.9185
Epoch  3/30 — Train Loss: 0.2099, Acc: 0.9372 | Val Loss: 0.1990, Acc: 0.9333
Epoch  4/30 — Train Loss: 0.1241, Acc: 0.9714 | Val Loss: 0.1893, Acc: 0.9370
Epoch  5/30 — Train Loss: 0.0997, Acc: 0.9769 | Val Loss: 0.1503, Acc: 0.9481
Epoch  6/30 — Train Loss: 0.0848, Acc: 0.9769 | Val Loss: 0.1315, Acc: 0.9593
Epoch  7/30 — Train Loss: 0.0826, Acc: 0.9769 | Val Loss: 0.1300, Acc: 0.9630
Epoch  8/30 — Train Loss: 0.0492, Acc: 0.9873 | Val Loss: 0.1157, Acc: 0.9667
Epoch  9/30 — Train Loss: 0.0394, Acc: 0.9920 | Val Loss: 0.1330, Acc: 0.9593
Epoch 10/30 — Train Loss: 0.0365, Acc: 0.9920 | Val Loss: 0.1162, Acc: 0.9593
Epoch 11/30 — Train Loss: 0.0308, Acc: 0.9936 | Val Loss: 0.1147, Acc: 0.9741
Epoch 12/30 — Train Loss: 0.0251, Acc: 0.9944 | Val Loss: 0.1266, Acc: 0.9556
Epoch 13/30 — Train Loss: 0.0198, Acc: 0.9968 | Val Loss: 0.1587

In [247]:
# 5) 최종 테스트 평가
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        test_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        test_labels.extend(y_batch.cpu().numpy())

print("\n=== Test Set Performance ===")
print(classification_report(test_labels, test_preds))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_preds))


=== Test Set Performance ===
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        23
           1       0.90      1.00      0.95        18
           2       0.96      0.96      0.96        26
           3       1.00      0.96      0.98        24
           4       1.00      1.00      1.00        37
           5       0.91      0.94      0.93        33
           6       1.00      0.97      0.98        30
           7       1.00      0.96      0.98        26
           8       1.00      0.89      0.94        19
           9       0.97      1.00      0.99        34

    accuracy                           0.97       270
   macro avg       0.97      0.97      0.97       270
weighted avg       0.97      0.97      0.97       270

Confusion Matrix:
 [[23  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  1 25  0  0  0  0  0  0  0]
 [ 0  0  0 23  0  1  0  0  0  0]
 [ 0  0  0  0 37  0  0  0  0  0]
 [ 0  0  1  0  0 31  0  0  